In [1]:
'''Importing basic data analysis packages'''
import numpy as np
import pandas as pd
import csv
import warnings
import os
import time
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import linear_model
import re
warnings.filterwarnings('ignore')

'''Plotting packages'''""
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.3)

## Read in previously created baseline CSV file

In [2]:
df = pd.read_csv('~/Desktop/w207/Project/Data/baseline_fixed.csv',sep=';')
print('Rows in dataset = ',len(df))

Rows in dataset =  1010392


In [3]:
df.head()

,employee_count,category_groups_list,country_code,uuid,p1_tag,rank,employee_count_ord,total_funding_usd,ind_1,ind_2,...,ind_37,ind_38,ind_39,ind_40,ind_41,ind_42,ind_43,ind_44,ind_45,ind_46
0,51-100,"Content and Publishing,Internet Services,Media...",USA,e1393508-30ea-8a36-3f96-dd3226033abd,0,102736.0,3.0,3.975000e+07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001-5000,"Information Technology,Internet Services,Priva...",USA,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,24985.0,7.0,NaN,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51-100,"Internet Services,Media and Entertainment",USA,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,9319.0,3.0,4.900000e+07,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101-250,"Financial Services,Lending and Investments,Sof...",USA,f4d5ab44-058b-298b-ea81-380e6e9a8eec,0,29637.0,4.0,NaN,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000+,"Apps,Commerce and Shopping,Content and Publish...",USA,df662812-7f97-0b43-9d3e-12f64f504fbb,0,222.0,9.0,2.335700e+09,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Check which variables have missing values and how many of the values are missing

In [4]:
num_missing = (df.isna().sum()).to_string()
print('NUMBER OF MISSING VALUES:\n\n',num_missing)

NUMBER OF MISSING VALUES:

 employee_count          291590
category_groups_list     87486
country_code            183072
uuid                         0
p1_tag                       0
rank                         1
employee_count_ord           0
total_funding_usd       886760
ind_1                    87486
ind_2                    87486
ind_3                    87486
ind_4                    87486
ind_5                    87486
ind_6                    87486
ind_7                    87486
ind_8                    87486
ind_9                    87486
ind_10                   87486
ind_11                   87486
ind_12                   87486
ind_13                   87486
ind_14                   87486
ind_15                   87486
ind_16                   87486
ind_17                   87486
ind_18                   87486
ind_19                   87486
ind_20                   87486
ind_21                   87486
ind_22                   87486
ind_23                   87486
ind_24     

To check for any obviously unusual values, we look at a simple distribution for each variable:

In [5]:
df.describe()

,p1_tag,rank,employee_count_ord,total_funding_usd,ind_1,ind_2,ind_3,ind_4,ind_5,ind_6,...,ind_37,ind_38,ind_39,ind_40,ind_41,ind_42,ind_43,ind_44,ind_45,ind_46
count,1.010392e+06,1.010391e+06,1.010392e+06,1.236320e+05,922906.000000,922906.000000,922906.000000,922906.000000,922906.000000,922906.000000,...,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000,922905.000000
mean,7.691074e-03,4.975178e+05,1.312830e+00,3.020067e+07,0.271362,0.175211,0.156251,0.056798,0.115233,0.120684,...,0.014371,0.009959,0.028268,0.016876,0.015634,0.010606,0.025314,0.006472,0.009662,0.019326
std,8.736091e-02,2.957022e+05,2.069258e+00,4.070341e+08,0.444663,0.380147,0.363093,0.231456,0.319303,0.325760,...,0.119014,0.099296,0.165739,0.128807,0.124056,0.102436,0.157076,0.080188,0.097819,0.137668
min,0.000000e+00,1.000000e+00,-1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,2.421215e+05,-1.000000e+00,3.320338e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,4.847480e+05,1.000000e+00,2.000000e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000e+00,7.516775e+05,2.000000e+00,1.095038e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000e+00,1.075072e+06,9.000000e+00,1.000000e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Impute Total Funding Amount Using Linear Regression

In [6]:
#First impute all variables with a simple method (e.g. median), and then follow with regression imputation iteratively
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

variables = ['total_funding_usd','rank']

for i in variables:
    df['imp_' + i] = imputer.fit_transform(df[i].values.reshape(-1,1))

#Remove ind columns for now
df_subset = df[['uuid', 'p1_tag', 'rank', 'total_funding_usd', 'employee_count_ord', 'imp_total_funding_usd', 'imp_rank']]

In [7]:
lin_reg_data = pd.DataFrame(columns = ['lin_' + i for i in variables])

for i in variables:
    lin_reg_data['lin_' + i] = df_subset['imp_' + i]
    parameters = list(set(df_subset.columns) - set(variables) - {'uuid'} - {'imp_' + i})
    
    #Run linear regression to impute values
    model = linear_model.LinearRegression()
    model.fit(X = df_subset[parameters], y = df_subset['imp_' + i])
    
    #Save imputed values
    lin_reg_data.loc[df_subset[i].isna(), 'lin_' + i] = model.predict(df_subset[parameters])[df_subset[i].isna()]

In [8]:
lin_reg_data.head()

,lin_total_funding_usd,lin_rank
0,3.975000e+07,102736.0
1,2.724724e+07,24985.0
2,4.900000e+07,9319.0
3,1.844520e+07,29637.0
4,2.335700e+09,222.0


In [9]:
#Add linear regression-imputed total_funding_usd and rank to original baseline dataset
df = pd.merge(df,lin_reg_data,how="inner",left_index=True,right_index=True)
df.drop(['imp_total_funding_usd','total_funding_usd','rank','imp_rank'], inplace=True, axis=1)
df = df.rename(columns={'lin_total_funding_usd':'total_funding_usd','lin_rank':'rank'})

In [10]:
#Check to make sure the right number of rows remain after merge
print('Rows in dataset = ',len(df))

Rows in dataset =  1010392


In [11]:
#Save to CSV
df.to_csv('~/Desktop/w207/Project/Data/baseline_2.csv',index=False,sep=';')

## Use Logistic Regression to Impute Industry Binary Variables

In [12]:
#First impute all variables with a simple method (in this case, just set to 0), and then follow with regression imputation iteratively
ind = ['ind_1', 'ind_2', 'ind_3', 'ind_4', 'ind_5', 'ind_6', 'ind_7', 'ind_8', 'ind_9', 'ind_10', 'ind_11', 'ind_12', 'ind_13',
       'ind_14', 'ind_15', 'ind_16', 'ind_17', 'ind_18', 'ind_19', 'ind_20', 'ind_21', 'ind_22', 'ind_23', 'ind_24', 'ind_25',
       'ind_26', 'ind_27', 'ind_28', 'ind_29', 'ind_30', 'ind_31', 'ind_32', 'ind_33', 'ind_34', 'ind_35', 'ind_36', 'ind_37',
       'ind_38', 'ind_39', 'ind_40', 'ind_41', 'ind_42', 'ind_43', 'ind_44', 'ind_45', 'ind_46']

for i in ind:
    df['imp_' + i] = df[i].fillna(0)

In [13]:
#Subset to relevant variables for regression
df_2_subset = df.drop(['employee_count','category_groups_list','country_code'], axis = 1)
num_columns = ['p1_tag','rank','employee_count_ord','total_funding_usd']

log_reg_data = pd.DataFrame(columns = ['log_' + i for i in ind])

for i in ind:
    log_reg_data['log_' + i] = df_2_subset['imp_' + i]
    parameters = list(set(df_2_subset.columns) - set(ind) - {'uuid'} - {'imp_' + i})
    
    #Run logisitic regression to impute values
    model = linear_model.LogisticRegression()
    model.fit(X = df_2_subset[parameters], y = df_2_subset['imp_' + i])
    
    #Save imputed values
    log_reg_data.loc[df_2_subset[i].isna(), 'log_' + i] = model.predict(df_2_subset[parameters])[df_2_subset[i].isna()]

In [14]:
#Add logistic regression-imputed variables to original baseline dataset
df = pd.merge(df, log_reg_data, how="inner",left_index=True,right_index=True)

#Drop original industry columns and columns with basic imputation (set na to 0)
imp = ['imp_ind_1', 'imp_ind_2', 'imp_ind_3', 'imp_ind_4', 'imp_ind_5', 'imp_ind_6', 'imp_ind_7', 'imp_ind_8', 'imp_ind_9',
       'imp_ind_10', 'imp_ind_11', 'imp_ind_12', 'imp_ind_13', 'imp_ind_14', 'imp_ind_15', 'imp_ind_16', 'imp_ind_17', 'imp_ind_18',
       'imp_ind_19', 'imp_ind_20', 'imp_ind_21', 'imp_ind_22', 'imp_ind_23', 'imp_ind_24', 'imp_ind_25', 'imp_ind_26', 'imp_ind_27',
       'imp_ind_28', 'imp_ind_29', 'imp_ind_30', 'imp_ind_31', 'imp_ind_32', 'imp_ind_33', 'imp_ind_34', 'imp_ind_35', 'imp_ind_36',
       'imp_ind_37', 'imp_ind_38', 'imp_ind_39', 'imp_ind_40', 'imp_ind_41', 'imp_ind_42', 'imp_ind_43', 'imp_ind_44', 'imp_ind_45',
       'imp_ind_46']

df.drop(imp, inplace=True, axis=1)
df.drop(ind, inplace=True, axis = 1)

def log_rename(col_name):
    if re.match(r"^log_", col_name):
        return (col_name[4:])
    else:
        return col_name

df.rename(columns=log_rename)

,employee_count,category_groups_list,country_code,uuid,p1_tag,employee_count_ord,total_funding_usd,rank,ind_1,ind_2,...,ind_37,ind_38,ind_39,ind_40,ind_41,ind_42,ind_43,ind_44,ind_45,ind_46
0,51-100,"Content and Publishing,Internet Services,Media...",USA,e1393508-30ea-8a36-3f96-dd3226033abd,0,3.0,3.975000e+07,102736.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001-5000,"Information Technology,Internet Services,Priva...",USA,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,7.0,2.724724e+07,24985.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51-100,"Internet Services,Media and Entertainment",USA,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,3.0,4.900000e+07,9319.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101-250,"Financial Services,Lending and Investments,Sof...",USA,f4d5ab44-058b-298b-ea81-380e6e9a8eec,0,4.0,1.844520e+07,29637.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000+,"Apps,Commerce and Shopping,Content and Publish...",USA,df662812-7f97-0b43-9d3e-12f64f504fbb,0,9.0,2.335700e+09,222.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010387,11-50,"Financial Services,Software",USA,c65bba1b-0bea-437c-a4fb-cd66540f0127,0,2.0,8.053359e+06,441163.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010388,11-50,"Administrative Services,Sports",USA,43f1b65f-9db9-4678-a16c-e19b303b21c5,0,2.0,6.126665e+06,615111.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010389,NaN,Transportation,USA,7f23e9b9-adf8-4ee4-869c-c73a555b399d,0,-1.0,-4.703136e+06,802836.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010390,NaN,Real Estate,USA,5362b44c-3b4b-401d-a1f7-eda569ca6d96,0,-1.0,-2.341167e+06,589590.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#Check to make sure the right number of rows remain after merge
print('Rows in dataset = ',len(df))
df.head()

Rows in dataset =  1010392


,employee_count,category_groups_list,country_code,uuid,p1_tag,employee_count_ord,total_funding_usd,rank,log_ind_1,log_ind_2,...,log_ind_37,log_ind_38,log_ind_39,log_ind_40,log_ind_41,log_ind_42,log_ind_43,log_ind_44,log_ind_45,log_ind_46
0,51-100,"Content and Publishing,Internet Services,Media...",USA,e1393508-30ea-8a36-3f96-dd3226033abd,0,3.0,3.975000e+07,102736.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001-5000,"Information Technology,Internet Services,Priva...",USA,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,0,7.0,2.724724e+07,24985.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51-100,"Internet Services,Media and Entertainment",USA,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,3.0,4.900000e+07,9319.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101-250,"Financial Services,Lending and Investments,Sof...",USA,f4d5ab44-058b-298b-ea81-380e6e9a8eec,0,4.0,1.844520e+07,29637.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000+,"Apps,Commerce and Shopping,Content and Publish...",USA,df662812-7f97-0b43-9d3e-12f64f504fbb,0,9.0,2.335700e+09,222.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Check number of missing values
num_missing = (df.isna().sum()).to_string()
print('NUMBER OF MISSING VALUES:\n\n',num_missing)

NUMBER OF MISSING VALUES:

 employee_count          291590
category_groups_list     87486
country_code            183072
uuid                         0
p1_tag                       0
employee_count_ord           0
total_funding_usd            0
rank                         0
log_ind_1                    0
log_ind_2                    0
log_ind_3                    0
log_ind_4                    0
log_ind_5                    0
log_ind_6                    0
log_ind_7                    0
log_ind_8                    0
log_ind_9                    0
log_ind_10                   0
log_ind_11                   0
log_ind_12                   0
log_ind_13                   0
log_ind_14                   0
log_ind_15                   0
log_ind_16                   0
log_ind_17                   0
log_ind_18                   0
log_ind_19                   0
log_ind_20                   0
log_ind_21                   0
log_ind_22                   0
log_ind_23                   0
log_ind_24 

In [17]:
#Save to CSV
df.to_csv('~/Desktop/w207/Project/Data/baseline_3_imputed.csv',index=False,sep=';')